In [1]:
# procedure for calculating hydraulic confinement along a pressure tunnel 
#  calculation done at stationed points at a selected interval along tunnel alignment
#  calculation determines the minimum dstance from the stationed point to the terrain surface 

In [2]:
# # python setup for qgis processing (based on TunnelGIS configuration)
# ##import sys
# from qgis.core import *
# from PyQt4.QtGui import *

# app = QApplication([], True)
# QgsApplication.setPrefixPath("/usr", True)
# QgsApplication.initQgis()

# sys.path.append('/usr/share/qgis/python/plugins')
# from processing.core.Processing import Processing
# Processing.initialize()
# from processing.tools import *

# import processing

In [3]:
# import required python libraires
import pandas as pd
import numpy as np
from numpy import *
import geopandas as gpd
import shapely as sp
import os 

import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# set wd for this procedure 
os.chdir("/home/kaelin_joseph/DSS.HydraulicConfinement/")

In [5]:
# check gpd version
gpd.__version__

u'0.3.0'

In [6]:
# check pd version
pd.__version__

u'0.22.0'

In [7]:
# define required input files
DTM = "data/swissALTI3D_.tif"  #DEM with surface topography
Alignment = "data/TestAlignment.p0.csv"  #tunnel alignment

In [8]:
# temporary data
Alignment_SHP ='tmp/TestAlignment.shp'  #alignment shp from Alignment
Alignment_grass = 'tmp/TestAlignmentGrass.csv'  #alignment csv fixed for grass
Alignment_line = "tmp/TestAlignmentLine.shp"  #intermediate data
#Alignment_points = "tmp/TestAlignmentPoints.shp"  #alignment containing station points
Alignment_points = "test/Line_Point.csv"  #alignment containing station points
Alignment_DTM = "tmp/TestAlignmentDTM.csv"  #alignment including terrain elevations at station points
Buffer_SHP = "tmp/TestBuffer.shp"  #buffer shp containing ring grid points at a particular station point
Test_buffer_csv = "tmp/TestBuffer.csv"  #buffer written to csv
Test_buffer_csv = "tmp/TestBuffer.csv"  #buffer written to csv


In [9]:
# define output files

In [10]:
# create alignment_df (dataframe) from alignment csv                         #ToDo JK: make into csv -> df function
#   required alignment input data: x_align, y_align, z_align at alignment definition points
#   check csv data before creating df (no trailing blank lines, no duplicate lines)
alignment_df = pd.read_csv(Alignment)
# delete row if only NA are present in row
alignment_df = alignment_df.dropna(how = "all")
# round alignment_df to three decimals
alignment_df = alignment_df.round(decimals=3)

In [11]:
alignment_df

,Point,Type,Station,Northing,Easting,Elevation
0,0,NaN,204+874.1,1269294.122,2612071.548,270.962
1,221,PT,205+389.861,1269029.594,2612426.331,253.058
2,EC10:3,EC,209+528.128,1265960.779,2614558.633,268.579


In [12]:
# create alignment_df (dataframe) from alignment csv                         #ToDo JK: make into csv -> df function
#   required alignment input data: x_align, y_align, z_align at alignment definition points
#   check csv data before creating df (no trailing blank lines, no duplicate lines)
alignment_points_df = pd.read_csv(Alignment_points)
# delete row if only NA are present in row
alignment_points_df = alignment_points_df.dropna(how = "all")
# round alignment_df to three decimals
alignment_points_df = alignment_points_df.round(decimals=3)

In [13]:
alignment_points_df.head()

,X,Y,cat_,Unnamed: 3
0,2612071.548,1269294.122,1,NaN
1,2612142.505,1269241.216,1,NaN
2,2612213.461,1269188.311,1,NaN
3,2612284.418,1269135.405,1,NaN
4,2612355.374,1269082.499,1,NaN


In [14]:
# Add variable "ID" to alignment_points_df
alignment_points_df["ID"] = alignment_points_df.index ### KLK: should be named id_align
alignment_points_df.head() 

,X,Y,cat_,Unnamed: 3,ID
0,2612071.548,1269294.122,1,NaN,0
1,2612142.505,1269241.216,1,NaN,1
2,2612213.461,1269188.311,1,NaN,2
3,2612284.418,1269135.405,1,NaN,3
4,2612355.374,1269082.499,1,NaN,4


In [15]:
# # create segmented polyline representing alignment from points in alignment_df
# #   grass input for function v.in.line 1) nust have x first and y second; 2) no header 
# #   required output data: x_align, y_align at each station point            #ToDo JK: make into stationing function
# alignment_df_grass = alignment_df.loc[:,["Easting", "Northing"]]  #x first and y second
# alignment_df_grass.to_csv(Alignment_grass, header=False, index=False)  #no header

# # points to line
# processing.runalg("grass7:v.in.lines",Alignment_grass,",",False,
#                   "2605158.67928,2621471.50192,1265877.44552,1269377.45567",0,Alignment_line)

# # line to station points
# processing.runalg("grass7:v.to.points",Alignment_line,"100",1,True,
#                   "2603510,2624270,1260650,1274890",-1,0.0001,0,Alignment_points)

In [16]:
# # create alignment_SHP from alignment_df for checking                       #ToDo JK: make into df -> shp function
# alignment_spatial_points = [sp.geometry.Point(row['Easting'], row['Northing'])
#                             for key, row in alignment_df.iterrows()]
# alignment_crs = {'init': 'epsg:2056'}  #define crs
# alignment_spatial = gpd.GeoDataFrame(alignment_df, geometry=alignment_spatial_points, crs = alignment_crs)
# alignment_spatial.to_file(Alignment_SHP, driver='ESRI Shapefile') 

# #print(alignment_spatial)

In [17]:
# # plot alignment for checking
# #   reference: http://nbviewer.jupyter.org/gist/perrygeo/c426355e40037c452434
# pylab.rcParams['figure.figsize'] = 13.5, 9

# # get data to plot
# line = gpd.GeoDataFrame.from_file(Alignment_line)
# point1 = gpd.GeoDataFrame.from_file(Alignment_SHP)
# point2 = gpd.GeoDataFrame.from_file(Alignment_points)

# # plot
# base = line.plot(color='black')
# point1.plot(ax=base, marker='o', color='red', markersize=10,  markeredgewidth=0.0);
# point2.plot(ax=base, marker='*', color='blue', markersize=10, markeredgewidth=0.0);

In [18]:
# # create alignment_points_df from Alignment_points shp                       #ToDo JK: make into shp -> df function
# #   required output data: id_align, x_align, y_align at each station point
# fp = Alignment_points
# alignment_points_df = gpd.read_file(fp)
# #type(alignment_points_df)
# #alignment_points_df.head()

# # create columns for Easting and Northing, then delete columns cat_ and geometry                          #ToDo KLK

# alignment_points_df.geometry
# #alignment_points_df.apply(lambda p: p.x)
# #alignment_points_df.apply(lambda p: p.y)

# alignment_points_df.head()



In [19]:
# s = gpd.GeoSeries([Point(4.42, 50.4), Point(4.43, 50.2)])
# s.apply(lambda p: p.x)

In [20]:
# interpolate alignment elevations at all station points and write to alignment_points_df                 #ToDo KLK
#   join alignment_points_df with alignment_df
#   required output data: z_align at each station point                #ToDo JK: make into z interpolation function

In [21]:
# Add variable "Distance_alignment" to alignment_df

alignment_df["Distance_alignment"] = np.nan

for n in range(0, len(alignment_df)-1): # why len(alignment_d -1? Because the distance cannot be calculated for the last point
    #print (n)
    alignment_df.iloc[n, alignment_df.columns.get_loc("Distance_alignment")] = ((alignment_df.iloc[n+1]["Easting"]-alignment_df.iloc[n]["Easting"])**2 +(alignment_df.iloc[n+1]["Northing"]-alignment_df.iloc[n]["Northing"])**2 )**(0.5)

alignment_df   

,Point,Type,Station,Northing,Easting,Elevation,Distance_alignment
0,0,NaN,204+874.1,1269294.122,2612071.548,270.962,442.544958
1,221,PT,205+389.861,1269029.594,2612426.331,253.058,3736.888722
2,EC10:3,EC,209+528.128,1265960.779,2614558.633,268.579,NaN


In [22]:
# Add variable "Distance_station" to alignment_points_df
alignment_points_df["Distance_station"] = np.nan

for n in range(0, len(alignment_points_df)-1): # why len(alignment_d -1? Because the distance cannot be calculated for the last point
    #print(n)
    alignment_points_df.iloc[n, alignment_points_df.columns.get_loc("Distance_station")] = ((alignment_points_df.iloc[n+1]["X"]-alignment_points_df.iloc[n]["X"])**2 +(alignment_points_df.iloc[n+1]["Y"]-alignment_points_df.iloc[n]["Y"])**2 )**(0.5)

alignment_points_df.head()

,X,Y,cat_,Unnamed: 3,ID,Distance_station
0,2612071.548,1269294.122,1,NaN,0,88.509551
1,2612142.505,1269241.216,1,NaN,1,88.508152
2,2612213.461,1269188.311,1,NaN,2,88.509551
3,2612284.418,1269135.405,1,NaN,3,88.508750
4,2612355.374,1269082.499,1,NaN,4,88.508954


In [23]:
# Add variable "Distance_station_sum" to alignment_points_df
alignment_points_df["Distance_station_sum"] = np.nan

for n in range(0, len(alignment_points_df)-1): # why len(alignment_d -1? Because the distance cannot be calculated for the last point
    id_sel2 = alignment_points_df.loc[(alignment_points_df.ID.isin(range(0,n+1))), "Distance_station"] 
    id_sel2= id_sel2.tolist()
    alignment_points_df.iloc[n, alignment_points_df.columns.get_loc("Distance_station_sum")] = sum(id_sel2)

alignment_points_df.head()

,X,Y,cat_,Unnamed: 3,ID,Distance_station,Distance_station_sum
0,2612071.548,1269294.122,1,NaN,0,88.509551,88.509551
1,2612142.505,1269241.216,1,NaN,1,88.508152,177.017703
2,2612213.461,1269188.311,1,NaN,2,88.509551,265.527255
3,2612284.418,1269135.405,1,NaN,3,88.508750,354.036004
4,2612355.374,1269082.499,1,NaN,4,88.508954,442.544958


In [24]:
# join alignment_df to alignment_points_df
alignment_points_df = pd.merge(left= alignment_points_df, right = alignment_df, 
                 left_on = ["X","Y"], 
                 right_on = ["Easting","Northing"], how = "left")

alignment_points_df.head()

,X,Y,cat_,Unnamed: 3,ID,Distance_station,Distance_station_sum,Point,Type,Station,Northing,Easting,Elevation,Distance_alignment
0,2612071.548,1269294.122,1,NaN,0,88.509551,88.509551,0,NaN,204+874.1,1269294.122,2612071.548,270.962,442.544958
1,2612142.505,1269241.216,1,NaN,1,88.508152,177.017703,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2612213.461,1269188.311,1,NaN,2,88.509551,265.527255,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2612284.418,1269135.405,1,NaN,3,88.508750,354.036004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2612355.374,1269082.499,1,NaN,4,88.508954,442.544958,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# clean up alignment_points_df
alignment_points_df = alignment_points_df.drop(columns =["cat_", "Unnamed: 3", "Point", "Type", "Northing", "Easting"])
alignment_points_df  

,X,Y,ID,Distance_station,Distance_station_sum,Station,Elevation,Distance_alignment
0,2612071.548,1269294.122,0,88.509551,88.509551,204+874.1,270.962,442.544958
1,2612142.505,1269241.216,1,88.508152,177.017703,NaN,NaN,NaN
2,2612213.461,1269188.311,2,88.509551,265.527255,NaN,NaN,NaN
3,2612284.418,1269135.405,3,88.508750,354.036004,NaN,NaN,NaN
4,2612355.374,1269082.499,4,88.508954,442.544958,NaN,NaN,NaN
5,2612426.331,1269029.594,5,98.339640,540.884598,205+389.861,253.058,3736.888722
6,2612482.444,1268948.835,6,98.338819,639.223417,NaN,NaN,NaN
7,2612538.557,1268868.077,7,98.338819,737.562236,NaN,NaN,NaN
8,2612594.670,1268787.319,8,98.340211,835.902447,NaN,NaN,NaN
9,2612650.784,1268706.560,9,98.338819,934.241266,NaN,NaN,NaN


In [26]:
# get ID of alignment_points_df where points of alignment_points_df == alignment_df
id_sel =  alignment_points_df.loc[(alignment_points_df.Elevation.isin(alignment_df["Elevation"])), "ID"] 
#alignment_points_df.loc[(alignment_points_df["Elevation"] == alignment_df["Elevation"][2]), "ID"] 
id_sel= id_sel.tolist()
id_sel

[0, 5, 43]

In [27]:
# Fill in "Elevation" and "Distance_alignment" for points of alignment_points_df where points of 
# alignment_points_df != alignment_df
# Maybe this step can be left out

for n in range(0, len(id_sel)-1): 
    alignment_points_df.loc[(alignment_points_df.ID.isin(range(id_sel[n]+1, id_sel[n+1]))), "Elevation"] = alignment_df["Elevation"][n]
alignment_points_df

for n in range(0, len(id_sel)-1): 
    alignment_points_df.loc[(alignment_points_df.ID.isin(range(id_sel[n]+1, id_sel[n+1]))), "Distance_alignment"] = alignment_df["Distance_alignment"][n]
alignment_points_df


,X,Y,ID,Distance_station,Distance_station_sum,Station,Elevation,Distance_alignment
0,2612071.548,1269294.122,0,88.509551,88.509551,204+874.1,270.962,442.544958
1,2612142.505,1269241.216,1,88.508152,177.017703,NaN,270.962,442.544958
2,2612213.461,1269188.311,2,88.509551,265.527255,NaN,270.962,442.544958
3,2612284.418,1269135.405,3,88.508750,354.036004,NaN,270.962,442.544958
4,2612355.374,1269082.499,4,88.508954,442.544958,NaN,270.962,442.544958
5,2612426.331,1269029.594,5,98.339640,540.884598,205+389.861,253.058,3736.888722
6,2612482.444,1268948.835,6,98.338819,639.223417,NaN,253.058,3736.888722
7,2612538.557,1268868.077,7,98.338819,737.562236,NaN,253.058,3736.888722
8,2612594.670,1268787.319,8,98.340211,835.902447,NaN,253.058,3736.888722
9,2612650.784,1268706.560,9,98.338819,934.241266,NaN,253.058,3736.888722


In [28]:
# Add variable "Distance_station_sum2" to alignment_points_df

alignment_points_df["Distance_station_sum2"] = np.nan

for n in range(1, len(id_sel)-1): 
    alignment_points_df.loc[(alignment_points_df.ID.isin(range(id_sel[n], id_sel[n+1]))), "Distance_station_sum2"] = alignment_points_df["Distance_station_sum"] - alignment_df["Distance_alignment"][n-1]

for n in range(0, 1): 
    alignment_points_df.loc[(alignment_points_df.ID.isin(range(id_sel[n], id_sel[n+1]))), "Distance_station_sum2"] =  alignment_points_df["Distance_station_sum"]
    
alignment_points_df

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


,X,Y,ID,Distance_station,Distance_station_sum,Station,Elevation,Distance_alignment,Distance_station_sum2
0,2612071.548,1269294.122,0,88.509551,88.509551,204+874.1,270.962,442.544958,88.509551
1,2612142.505,1269241.216,1,88.508152,177.017703,NaN,270.962,442.544958,177.017703
2,2612213.461,1269188.311,2,88.509551,265.527255,NaN,270.962,442.544958,265.527255
3,2612284.418,1269135.405,3,88.508750,354.036004,NaN,270.962,442.544958,354.036004
4,2612355.374,1269082.499,4,88.508954,442.544958,NaN,270.962,442.544958,442.544958
5,2612426.331,1269029.594,5,98.339640,540.884598,205+389.861,253.058,3736.888722,98.339640
6,2612482.444,1268948.835,6,98.338819,639.223417,NaN,253.058,3736.888722,196.678459
7,2612538.557,1268868.077,7,98.338819,737.562236,NaN,253.058,3736.888722,295.017278
8,2612594.670,1268787.319,8,98.340211,835.902447,NaN,253.058,3736.888722,393.357489
9,2612650.784,1268706.560,9,98.338819,934.241266,NaN,253.058,3736.888722,491.696308


In [29]:
# Add variable "Z" to alignment_points_df


alignment_points_df["Z"] = np.nan

for i in range(0, len(alignment_points_df)): # why len(alignment_d -1? Because the distance cannot be calculated for the last point
    if i in id_sel:
        alignment_points_df.iloc[i, alignment_points_df.columns.get_loc("Z")] = alignment_points_df.iloc[i]["Elevation"]

    else:
        
        id_sel2 = id_sel
        id_sel2.append(i)
        id_sel2.sort()
        
        m = id_sel2.index(i)+1     
        n = id_sel2[m]  
        o = id_sel2.index(i)-1
        p = id_sel2[o]
        
        alignment_points_df.iloc[i, alignment_points_df.columns.get_loc("Z")] = alignment_points_df.iloc[p]["Elevation"]+(alignment_points_df.iloc[n]["Elevation"]-alignment_points_df.iloc[p]["Elevation"])/alignment_points_df.iloc[i]["Distance_alignment"]*alignment_points_df.iloc[i-1]["Distance_station_sum2"]
    
#         print("i:",i)
#         print(id_sel2)
#         print("m:", m)
#         print("n:",n)
#         print("o:", o)
#         print("p:",p)
#         print("Elevation n:",alignment_points_df.iloc[n]["Elevation"])
#         #print("Elevation Test:", alignment_df.loc[(alignment_df["ID"] == n), "Elevation"]) # maybe above mentioned code can be replaced through this
#         print("Elevation n:",alignment_points_df.iloc[p]["Elevation"])
#         print("\n")
        
        id_sel2.remove(i)

alignment_points_df

,X,Y,ID,Distance_station,Distance_station_sum,Station,Elevation,Distance_alignment,Distance_station_sum2,Z
0,2612071.548,1269294.122,0,88.509551,88.509551,204+874.1,270.962,442.544958,88.509551,270.962000
1,2612142.505,1269241.216,1,88.508152,177.017703,NaN,270.962,442.544958,177.017703,267.381177
2,2612213.461,1269188.311,2,88.509551,265.527255,NaN,270.962,442.544958,265.527255,263.800411
3,2612284.418,1269135.405,3,88.508750,354.036004,NaN,270.962,442.544958,354.036004,260.219589
4,2612355.374,1269082.499,4,88.508954,442.544958,NaN,270.962,442.544958,442.544958,256.638798
5,2612426.331,1269029.594,5,98.339640,540.884598,205+389.861,253.058,3736.888722,98.339640,253.058000
6,2612482.444,1268948.835,6,98.338819,639.223417,NaN,253.058,3736.888722,196.678459,253.466449
7,2612538.557,1268868.077,7,98.338819,737.562236,NaN,253.058,3736.888722,295.017278,253.874895
8,2612594.670,1268787.319,8,98.340211,835.902447,NaN,253.058,3736.888722,393.357489,254.283341
9,2612650.784,1268706.560,9,98.338819,934.241266,NaN,253.058,3736.888722,491.696308,254.691793


In [30]:
# alignment_points_df to spatial

In [31]:
# # use grass7:r.what.points to get DTM value for z for points along alignment and write to .csv file
                                                                             #ToDo JK: make into what.points function

# alignment_points_df to alignment_points_shp

# processing.runalg(                                                       
#                   "grass7:r.what.points",DTM,alignment_points_shp,
#                   "NA",",",500,True,False,False,False,False,
#                   "2603510.0,2624270.0,1260650.0,1274890.0",-1,0.0001,Alignment_DTM
#                  )
# #out = open(Alignment_DTM, 'r')
# #print(out.read())  #check

In [32]:
# read Alignment_DTM csv and write to alignment_points_df             #ToDo JK: reuse function csv -> df from above
#   required output data: z_dtm at each station point





In [33]:
# # prepare circular grid as buffer around a station point and determine minimal distance to terrain surface 
# #   this calculation is defined as a function, to be applied at each station point
# #   inputs are easting, northing, overburden at stationed point (and a coeff for buffer_radius = coeff*cover) 

# # input data  #temporarily static
# point = np.array([2612060, 1268065])  #station point [from alignment_points_df]
# h = 110.0  #overburden depth above station point [calculate from alignment_points_df]

# def make_buffer(point, overburden):
#     c = 0.5  #buffer radius = c*h
#     res = 20.0  #nominal grid resolution 
#     intvls_r = int(h*c / res)  #number of intervals along the buffer radius, close enough
#     res_r = h*c / intvls_r  #effective resolution along the radius
#     buffer = np.array(point)  #initialize buffer, first item is exactly at station point
#     print("buffer: "+str(buffer))

#     # calculate local coordinates for grid along a ring
#     for i in range(intvls_r):
#         r = c*h - i*res_r
#         perim = 2 * r * pi 
#         intvls_c = int(perim/res)  #number of intervals along a ring, close enough
#         item = np.array([0.0, 0.0])  #initialize
#         for j in range(intvls_c):
#             item[0] = (sin((2*pi) / intvls_c *(j+1)) *r) + point[0]
#             item[1] = (cos((2*pi) / intvls_c *(j+1)) *r) + point[1]
#             buffer = np.vstack((buffer, item))  #vstack works with arrays of diff nr of items, append does not        

#         print("r: "+str(r))
#         print("intvls_r: "+str(intvls_r))
#         print("intvls_c: "+str(intvls_c))
#         return buffer
        
# buffer = make_buffer(point=point, overburden=h)        

In [34]:
# # use make_buffer to get buffer grid points at all station points along alignment                         #ToDo KLK
# #   write buffer points to buffer_df and to alignment_points_df
# #   required output data: id_align, id_buffer, x_buffer, y_buffer, z_dtm at each buffer point  

# # temporary code for handling one buffer
# # create Buffer_SHP for grass from buffer
# crs = {'init': 'epsg:2056'}  #define crs

# buffer_df = pd.DataFrame.from_records(data=buffer, columns=['Easting', 'Northing'], index=None)
# #buffer_df





In [35]:
# # create shp file with all buffer points                                         #ToDo JK: reuse df -> shp function
# buffer_points = [sp.geometry.Point(row['Easting'], row['Northing'])
#                  for key, row in buffer_df.iterrows()]
# #len(buffer_points)  #check

# # write buffer to Buffer_SHP as shape file
# buffer_spatial = gpd.GeoDataFrame(buffer_df, geometry=buffer_points, crs = crs)
# buffer_spatial.to_file(Buffer_SHP, driver='ESRI Shapefile') 
# #print(buffer_spatial)  #check
# print(Buffer_SHP)

In [36]:
# # use grass7:r.what.points to read Buffer_SHP to get z_dtm at each buffer point for all station points

# # temporary code for handling one buffer
# # use grass7:r.what.points to read Buffer_SHP to get z_dtm at each point
# test_buffer_csv = "tmp/test_buffer.csv"
# # buffer_SHP = {'init': 'epsg:2056'}  #define crs
# processing.runalg("grass7:r.what.points",DTM,Buffer_SHP,            #ToDo JK: reuse what.points function from above
#                   "NA",",",500,True,False,False,False,False,
#                   "2603510.0,2624270.0,1260650.0,1274890.0",-1,0.0001,test_buffer_csv
#                  )

# out = open(test_buffer_csv, 'r')
# print(out.read())  #check

In [37]:
# read buffer points from Test_buffer_csv into alignment_points_df
#   required output data: id_align, id_buffer, x_buffer, y_buffer, z_dtm at each buffer point  
                                                                      #ToDo JK: reuse function csv -> df from above





In [38]:
# calculate minimum distance to terrain in each buffer ring
#   required output data: dist for each buffer point, min_dist for buffer ring





In [39]:
# calculate hydraulic confinement safety factor at each station point
#   required input data: reference maximum water pressure elevation (static or dynamic ??) 





In [40]:
# plot results for hydraulic confinement safety factor as horizontal bar beneath longitudinal profile 



